In [1]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import math
import pywt
import itertools
from scipy import stats

pd.options.mode.use_inf_as_na = True

Using matplotlib backend: TkAgg


In [87]:
TimeDwellOrig = 800
TimeFixation = 300

In [88]:
ipaDifficult = list()
ipaMedium = list()
ipaEasy = list()

In [89]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [90]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [91]:
def OptiKeyTypingTime(UserKeys):
    
    TimeTyping = dict()
    
    time1, t1, t2 = UserKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = UserKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    TimeTyping['startTime'] = startTime
    TimeTyping['endTime'] = endTime
    
    return TimeTyping

In [92]:
def FindTrialEndTimes(KeysSelected, timeTyping):
    # function to find start and end of tasks in experiments
    timeStartEnd = list() # format of this list will be: [startTime1, endTime1/startTime2, endTime2/startTime3, ..., endTimeN]
    
    timeStartEnd.append(timeTyping['startTime'])
    
    nTrial = 1
    
    for keys in KeysSelected:
        
        if keys[1] == 'NextPhrase':
            time1, t1, t2 = keys[0].partition('+')
            endTimeTrial = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            timeStartEnd.append(endTimeTrial)
    
    
    timeStartEnd.append(timeTyping['endTime'])
    
    
    return timeStartEnd

In [93]:
def DwellTimeForBaseline(UserKeys_wDwellTime):
    
    DwellTime = list()
    
    for key in UserKeys_wDwellTime:
        if key[1] == 'NextPhrase':
            #print('NextPhrase found at ', key[2])
            if key[2] == 1:
                DwellTime.append(key[3])
                
    return DwellTime

In [94]:
# function to convert list of date and time into datetime format list
def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [95]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [96]:
def CreateTimeEpochsOfTrials(TimeStartEndMixed, UserKeys):
    # function to use list of mixed start and end times of trials and keys looked at by user to create trial epochs
    
    TimeEpochTrial = dict()
    TimeEpochTrial['Start'] = list()
    TimeEpochTrial['End'] = list()
    
    # Create list of times in userKeys to be able to use function 'nearestTimePoint'
    UserKeysStrTimes = [item3[0] for item3 in UserKeys]
    UserKeysTimes = timeConversion(UserKeysStrTimes)
    
    Flag_FoundSleepKey = 0 # Flag to indicate finding sleep key
    
    n = -1
    for time in TimeStartEndMixed:
        n = n + 1
        Flag_FoundSleepKey = 0
        
        if n == 0: # first time is only start time for the first trial
            TimeEpochTrial['Start'].append(time)
            continue
        elif n == len(TimeStartEndMixed)-1: # last time is only the end time for last trial
            
            TimeEpochTrial['End'].append(time)
            
        else: # the middle elements need to be divided into start and end
            TimeEpochTrial['End'].append(time)
            
            timeCheck = time
            
            # find the time in userkeys. Keep going to the previous element till you reach start of selection of
            # nextPhrase key
            while Flag_FoundSleepKey < 1:
                
                nearestToTrialStartTime, nearestToTrialStartInd = nearestTimePoint(UserKeysTimes, timeCheck)
                indCheck = nearestToTrialStartInd
                
                if 'NextPhrase' not in UserKeys[indCheck][1]:
                    TimeEpochTrial['Start'].append(nearestToTrialStartTime)
                    Flag_FoundSleepKey = 1
                    break
                else:
                    indCheck = indCheck - 2 # 2 added instead of 1, to allow nearestTimePoint to find the one before this
                    timeCheck = UserKeysTimes[indCheck]
                    
                
    return TimeEpochTrial      
            

In [97]:
def Convert2ColumnSizesTo1(GazeLog):
    # function to convert pupilsizes from 2 columns for every pupil due to comma use instead of decimal, 
    # to proper pupil sizes
    
    PupilLogL = list()
    PupilLogR = list()
    
    PupilLogL_beforeDecimal = [item4[-5] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    PupilLogL_afterDecimal = [item4[-4] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    PupilLogR_beforeDecimal = [item4[-2] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    PupilLogR_afterDecimal = [item4[-1] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    
    for i in range(0, len(PupilLogL_beforeDecimal)):
        if 'Valid' not in PupilLogL_beforeDecimal[i] and 'Valid' not in PupilLogL_afterDecimal[i]:
            if 'nan' not in PupilLogL_beforeDecimal[i] and 'nan' not in PupilLogL_afterDecimal[i]:
                PupilLogL.append(float(PupilLogL_beforeDecimal[i]+'.'+PupilLogL_afterDecimal[i]))
            else:
                PupilLogL.append(np.nan)
        else:
            # Rarely, the pupil size is a whole number
            PupilLogL.append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
            # right or left eye has whole number pupil size
    
    for i in range(0, len(PupilLogR_beforeDecimal)):
        if 'Valid' not in PupilLogR_beforeDecimal[i] and 'Valid' not in PupilLogR_afterDecimal[i]:
            if 'nan' not in PupilLogR_beforeDecimal[i] and 'nan' not in PupilLogR_afterDecimal[i]:
                PupilLogR.append(float(PupilLogR_beforeDecimal[i]+'.'+PupilLogR_afterDecimal[i]))
            else:
                PupilLogR.append(np.nan)
        else:
            # Rarely, the pupil size is a whole number
            PupilLogL.append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
            # right or left eye has whole number pupil size
            
    # if one of the pupils are nan, the other one is converted too
    nPupil = -1
    for pupilL in PupilLogL:
        nPupil = nPupil + 1
        if np.isnan(pupilL):
            if nPupil < len(PupilLogR):
                if not np.isnan(PupilLogR[nPupil]):
                    PupilLogR[nPupil] = np.nan
            else:
                del PupilLogL[len(PupilLogR):]
                
    nPupil = -1
    for pupilR in PupilLogR:
        nPupil = nPupil + 1
        if np.isnan(pupilR):
            if nPupil < len(PupilLogL):
                if not np.isnan(PupilLogL[nPupil]):
                    PupilLogL[nPupil] = np.nan
            else:
                del PupilLogR[len(PupilLogL):]
                
    #print(len(PupilLogL), len(PupilLogR))
    
    return PupilLogL, PupilLogR

In [98]:
def PupilSizeFromTrialTimes(TimeTrial, TimeGazeLog, TimeInternalGazeLog, PupilSizeLogL, PupilSizeLogR):
    # find pupil sizes from the start and end time given
    
    # find start and end time in gazeLog
    timeStart, timeStartInd = nearestTimePoint(TimeGazeLog, TimeTrial[0])
    timeEnd, timeEndInd = nearestTimePoint(TimeGazeLog, TimeTrial[1])
    
    pupilSize_TrialL = PupilSizeLogL[timeStartInd: timeEndInd]
    pupilSize_TrialR = PupilSizeLogR[timeStartInd: timeEndInd]
    
    TimeInternal_Trial = TimeInternalGazeLog[timeStartInd: timeEndInd]
    
    TimeGaze_Trial = TimeGazeLog[timeStartInd: timeEndInd]
    
    return pupilSize_TrialL, pupilSize_TrialR, TimeGaze_Trial, TimeInternal_Trial
    

In [99]:
def filterBlinks(pupilData, timeInDatetime_trial, timeInS_Trial):
    # filter any blinks and nan values lasting around 250ms (on average)
    # http://faculty.washington.edu/chudler/facts.html
   
    # recording extra blink information - duration and frequency
    blinkDurationList = list()
    blinkTimeList = list()
    blinkCount = 0
    nonBlinkCount = 0
    nonBlinkTimeList = list()
    timeRemove = 0
    
    # blink is every nan value in the range of 100-400ms 
    # 250 ms (23 samples) before and after the blink will also be removed
    extraBlinkSamples = 23    
    
    # remove single missing data, that are due to hardware error
    missingVal_Single = np.argwhere(np.isnan(pupilData))
    missingVal_Single = list(itertools.chain.from_iterable(missingVal_Single)) # flatten the list
    missingVal_SingleDifference = [t - s for s, t in zip(missingVal_Single, missingVal_Single[1:])] # find difference 
    # between consecutive elements
    missingVal_SingleDifference.insert(0, missingVal_Single[0]) # insert the first blink index in the beginning of list
    
    # the list missingVal_SingleDifference contains the index of the first blink, followed by the difference in the index to 
    # the next nan value
    
    # first remove the single nan values, which are missing data
    eyeTracker_missingData = list() # list with index of single missing data  
    valInd = -1

    for val in missingVal_SingleDifference:
        valInd = valInd + 1
        if valInd == 0:
            continue
        if val != 1:
            if missingVal_SingleDifference[valInd-1] !=1: # if there are 2 consecutive missing values (denoted by 2 consecutive
                # non 1 numbers, they are added to the list of eyeTracker_missingData)
                eyeTracker_missingData.append(sum(missingVal_SingleDifference[:valInd]))
                
    # remove single missing values from pupil data
    pupilData_woSingleMissingData0 = [pupilData[ind] if ind not in eyeTracker_missingData else [] 
                                     for ind in range(0, len(pupilData))]
    pupilData_woSingleMissingData = [x for x in pupilData_woSingleMissingData0 if x]
    
    # remove the times for single missing values in pupil data
    timeList_woSingleMissingData0 = [timeInDatetime_trial[ind] if ind not in eyeTracker_missingData else [] 
                                     for ind in range(0, len(timeInDatetime_trial))]
    timeList_woSingleMissingData = [x for x in timeList_woSingleMissingData0 if x]
    
#     print(len(timeList_woSingleMissingData))
    
    
    timeInS_woSingleMissingData = timeInS_Trial[-1]-(len(timeList_woSingleMissingData)-len(timeInDatetime_trial))/90
    #print(timeInS_woSingleMissingData, timeInS_Trial[-1])
    
    # find the nan values again from pupilData_woSingleMissingData
    missingVal_Rest = np.argwhere(np.isnan(pupilData_woSingleMissingData))
    missingVal_Rest = list(itertools.chain.from_iterable(missingVal_Rest))
    missingVal_RestDifference = [t - s for s, t in zip(missingVal_Rest, missingVal_Rest[1:])]
    missingVal_RestDifference.insert(0, missingVal_Rest[0])
    #print(missingVal_RestDifference)
    
    # compile and create list of start and end of blinks
    blink_missingData = dict()
    blink_missingData['Start'] = list()
    blink_missingData['End'] = list()
    
    valInd = -1
    for val in missingVal_RestDifference:
        valInd = valInd + 1
        if val > 1:
            
            
            #print('value', val)
            # instead of appending the actual index of blink start, since 250ms before and after the blink need to be
            # removed, it is also appended here.
            
            # just make sure that the additional samples do not make the index of blink go in negative
            if sum(missingVal_RestDifference[:valInd+1])-extraBlinkSamples > 0:
                
                blink_missingData['Start'].append(sum(missingVal_RestDifference[:valInd+1])-extraBlinkSamples)
            else:
                blink_missingData['Start'].append(0)
            
            if valInd == 0:
                lastBlinkStart = valInd
                continue
                
            # append blink duration list
            blinkDurationCurrent = valInd-lastBlinkStart
            # if blink duration is greater than 1s, it is not considered to be blink anymore
            if blinkDurationCurrent < 90: # since tobii sampling frequency is 90Hz
                blinkCount = blinkCount + 1
                blinkDurationList.append(blinkDurationCurrent/90)
                blinkTimeList.append(timeList_woSingleMissingData[sum(missingVal_RestDifference[:valInd+1])])
                lastBlinkStart = valInd
            else:
                # collect the time of non-blinks, that will need to be removed from trial time, to calculate 
                # blink frequency
                #print('current blink duration', valInd, lastBlinkStart, blinkDurationCurrent)
                timeRemove = timeRemove + blinkDurationCurrent
                nonBlinkCount = nonBlinkCount + 1
                nonBlinkTimeList.append(timeList_woSingleMissingData[sum(missingVal_RestDifference[:valInd+1])])
                lastBlinkStart = valInd
            
            # make sure that the additional samples do not increase the index to more than the length of the pupilData
            if sum(missingVal_RestDifference[:valInd])+extraBlinkSamples < len(pupilData_woSingleMissingData):
                blink_missingData['End'].append(sum(missingVal_RestDifference[:valInd])+extraBlinkSamples)
            else:
                blink_missingData['End'].append(len(pupilData_woSingleMissingData)-1)
#         else:
#             # val is 1
#             if valInd-2 > 0 and valInd+3 < len(missingVal_RestDifference):
#                 if missingVal_RestDifference[valInd-1] > 1:
#                     if missingVal_RestDifference[valInd+1] == 1:
#                         if missingVal_RestDifference[valInd+2] > 1:
#                             print(timeList_woSingleMissingData[sum(missingVal_RestDifference[:valInd+1])], 
#                                 missingVal_RestDifference[valInd-2:valInd+3])
#                             if missingVal_RestDifference[valInd+2] > missingVal_RestDifference[valInd-1]:
#                                 if valInd-6>0:
#                                     print(missingVal_RestDifference[valInd-6:valInd+3])
#                     elif missingVal_RestDifference[valInd+1] > 1:
#                         print(timeList_woSingleMissingData[sum(missingVal_RestDifference[:valInd+1])], 
#                               missingVal_RestDifference[valInd-2:valInd+3])
#                         if missingVal_RestDifference[valInd+2] > missingVal_RestDifference[valInd-1]:
#                                 if valInd-6>0:
#                                     print(missingVal_RestDifference[valInd-6:valInd+3])
                        
                        
    # add the last blink index
    # make sure that the additional samples do not increase the index to more than the length of the pupilData
    if sum(missingVal_RestDifference)+extraBlinkSamples < len(pupilData_woSingleMissingData):
        blink_missingData['End'].append(sum(missingVal_RestDifference)+extraBlinkSamples)
    else:
        blink_missingData['End'].append(len(pupilData_woSingleMissingData)-1)
                
    # need to create a list containing indexes that are to be removed
    blinkIndexList = list()
    
#     print(len(blink_missingData['Start']), len(blink_missingData['End']))
    
    # remove blinks and additional data from pupil data to get filtered data
    for indInd in range(0, len(blink_missingData['Start'])):
        blinkIndexList.append(range(blink_missingData['Start'][indInd], blink_missingData['End'][indInd]+1))
    # flatten the list
    blinkIndexList = list(itertools.chain.from_iterable(blinkIndexList))
    
    pupilData_woRestMissingData0 = [pupilData_woSingleMissingData[ind] if ind not in blinkIndexList else [] 
                                     for ind in range(0, len(pupilData_woSingleMissingData))]
    pupilData_filter = [x for x in pupilData_woRestMissingData0 if x]
    
    # remove the times for single missing values in pupil data
    timeList_woRestMissingData0 = [timeList_woSingleMissingData[ind] if ind not in blinkIndexList else [] 
                                     for ind in range(0, len(timeList_woSingleMissingData))]
    time_filter = [x for x in timeList_woRestMissingData0 if x]
    
    timeInS_Trial_filter = timeInS_woSingleMissingData-timeRemove/90
    
    blinkFrequency = blinkCount/timeInS_Trial_filter
    #print('freq', blinkFrequency, timeInS_woSingleMissingData, timeRemove)
    #print('time difference', len(timeInDatetime_trial), len(time_filter))
    if np.nan in pupilData_filter:
        print('nan values in filtered data')
#         for i in enumerate(pupilData_filter):
#             print(i)
        
    #print(nonBlinkCount, blinkCount, nonBlinkTimeList)
    return pupilData_filter, time_filter, blink_missingData, blinkDurationList, blinkFrequency, blinkTimeList, timeInS_Trial_filter

In [100]:
def modmax(d):
    # modulus maxima detection
    
    # compute signal modulus
    m = [0.0]*len(d)
    for i in range(0, len(d)):
        m[i] = math.fabs(d[i])
    
    # if value is larger than both neighbours , and strictly
    # larger than either , then it is a local maximum
    t = [0.0]*len(d)
    for i in range(0, len(d)):
        ll = m[i-1] if i >= 1 else m[i]
        oo = m[i]
        rr = m[i+1] if i < len(d)-2 else m[i]
        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
            # compute magnitude
            t[i] = math.sqrt(d[i]**2)
        else:
            t[i] = 0.0
    #print(len(t))
    return t

In [101]:
def plotPupilSize(pupilData, timeData, TrialNumber):
    
    dataLenEqualizer = min(min(len(pupilData['Left']), len(pupilData['Right'])), len(timeData))
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(timeData[0:dataLenEqualizer], pupilData['Left'][0:dataLenEqualizer], 'b')
    ax.plot(timeData[0:dataLenEqualizer], pupilData['Right'][0:dataLenEqualizer], 'r')
    
    ax.set_ylabel('Absolute pupil size [in mm]')

    ax.set_title(TrialNumber)


In [102]:
def hampel(vals_orig, k, sd):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''
    # Obtained from: https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-
    # hampel-function-faster
    
    #plt.plot(vals_orig)
    
    #Make copy so original not edited
    vals = pd.DataFrame(vals_orig)      
    #print(vals.isnull().any())
    vals0 = vals.replace([np.inf, -np.inf], np.nan)
    #vals = vals0.astype(float).fillna(method = 'backfill') # linear interpolation instead 
    #print(vals)
    vals = vals0.astype(float).interpolate('linear', limit_direction = 'both') # linear interpolation instead of 
    # simply copying the previous value --\ linear interpolation than cubic to not add any patterns in the data, limit direction
    # set to both, to interpolate the nan values occuring from the start of the series
    
    L= 1.4826
    rolling_median = vals.rolling(window=k, min_periods=1, center=True).median()
    
    #print(rolling_median)
    difference = np.abs(rolling_median-vals)
    median_abs_deviation = difference.rolling(k).median()
    threshold = sd * L * median_abs_deviation
    outlier_idx = difference>threshold
    vals[outlier_idx] = rolling_median[outlier_idx]
    #print(vals)
    #print('datatype', vals.dtypes)
    #print(vals.isnull().any())
    #vals.plot()
    return(vals)

In [103]:
def ipaFunc(d):
    # compute ipa value of pupil diameter
    IPA = list()
    #print(len(d.pupildata.values))
    # obtain 2-level DWT of pupil diameter signal d
    
    # get signal duration (in seconds)
    tt = ((d.timestamp.values[-1] - d.timestamp.values[0]).item())/1000000000

    
    
    try:
        (cA2,cD2,cD1) = pywt.wavedec(d.pupildata.values,'sym12','symmetric', 2)
    except ValueError:
        print('value error in wavedec')
        return
        
    # normalize by 1=2j , j = 2 for 2-level DWT
    cA2[:] = [x / math.sqrt(4.0) for x in cA2]
    cD1[:] = [x / math.sqrt(2.0) for x in cD1]
    cD2[:] = [x / math.sqrt(4.0) for x in cD2]
    
    # detect modulus maxima , see Listing 2
    cD2m = modmax(cD2)
    #print(len(cD2m))
    
    # threshold using universal threshold l_univ = s*sqrt(2logn)
    # where s is the standard deviation of the noise
    luniv = np.std(cD2m) * math.sqrt(2.0*np.log2(len(cD2m)))
    cD2t = pywt.threshold(cD2m ,luniv,mode="hard")
        
    # compute IPA
    ctr = 0
    for i in range(0, len(cD2t)):
        if math.fabs(cD2t[i]) > 0: ctr += 1
        #IPA = float(ctr)/tt
        # maybe each pupil data has an IPA?
    IPA = (float(ctr)/tt)
    
    return IPA, cD2m, cD2t, cD2, cD1, cA2, tt

In [106]:
def FindAndPlotPupilSizeForEpoch(GazeLog, TimeEpochTrial, ScoresDifficulty, DwellTimes_ForBaseline):
    # function that uses the list of start and end trial times to find the pupil sizes for those trials and plots them
    
    # first create a list of times in gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    # internal time, to depict seconds
    timeInternalGazeLog = [float(item3[1]) for item3 in GazeLog]
    
    # extract pupil sizes in decimals from the strange 2 columns for every pupil
    pupilLogL, pupilLogR = Convert2ColumnSizesTo1(GazeLog)
    
    ipaList = list()
    timeOfGaze_TrialList = list()
    
    
    blinkDurationList = list()
    blinkDurationAverageList = list()
    blinkTimeList = list()
    blinkFrequencyList = list()
    timeInS_List = list()
    pupilMean = list()
    
    # for every epoch, plot the pupil size
    for trialNr in range(0, len(timeEpochTrial['Start'])):
        if trialNr == 0:
            continue
        # find pupil sizes for the trial
        pupilSizeL_Trial, pupilSizeR_Trial, timeGaze_Trial, timeInternal_Trial = PupilSizeFromTrialTimes(
            [TimeEpochTrial['Start'][trialNr], TimeEpochTrial['End'][trialNr]], timeGazeLog, 
                                timeInternalGazeLog, pupilLogL, pupilLogR)
        
        pupilSize_Trial = dict()
        pupilSize_Filter = dict()
        pupilSize_woBlink = dict()
        
        # find difference in consecutive elements of internal time
        timeInternalDifference = [t - s for s, t in zip(timeInternal_Trial, timeInternal_Trial[1:])]
        # divide by 1000 to make it s
        timeOfGaze_Trial = [sum(timeInternalDifference[:i])/1000000 for i in range(1,len(timeInternalDifference))]

        # some trials were skipped, because the sentence was written before. If the time of trial is less than
        # 10s, the trial is skipped
        if timeOfGaze_Trial[-1] < 20:
            print('trial number ', trialNr+1, 'with', timeOfGaze_Trial[-1], 's will be skipped')
            continue
        
        pupilSize_Trial['Left'] = pupilSizeL_Trial
        pupilSize_Trial['Right'] = pupilSizeR_Trial
        
        #if trialNr == 4:
        #    for i in range(0, len(pupilSizeL_Trial)):
        #        print(pupilSizeL_Trial[i], pupilSizeR_Trial[i])
            
        #print('Trial', len(pupilSizeL_Trial), len(pupilSizeR_Trial))
        
        # filter the blinks
        pupilSizeL_woBlink, time_filter, missingPupilData, blinkDuration, blinkFrequency, blinkTimeList, timeInS_filter = filterBlinks(pupilSizeL_Trial, timeGaze_Trial, timeOfGaze_Trial)
        pupilSizeR_woBlink, time_filter, missingPupilData, blinkDuration, blinkFrequency, blinkTimeList, timeInS_filter = filterBlinks(pupilSizeR_Trial, timeGaze_Trial, timeOfGaze_Trial)
        
#         print(trialNr, blinkFrequency)
        #print(trialNr, blinkDuration)

        # time of trial
        timeInS_List.append(timeInS_filter)
        #print(trialNr, timeInS_filter)
        
        #print(index_blinkEndL)
        #print(index_blinkEndR)
        pupilSize_woBlink['Left'] = pupilSizeL_woBlink
        pupilSize_woBlink['Right'] = pupilSizeR_woBlink
        
        #print('After blink', len(pupilSizeL_woBlink), len(pupilSizeR_woBlink))
        # Hampel filter to remove the outliers
        winSize = 25
        pupilSizeL_filter = hampel(pupilSizeL_woBlink, winSize, 3)
        pupilSizeR_filter = hampel(pupilSizeR_woBlink, winSize, 3)

        pupilSize_Filter['Left'] = pupilSizeL_filter.values.tolist()
        pupilSize_Filter['Right'] = pupilSizeR_filter.values.tolist()
        
        pupilSizeL_filterList = [i[0] for i in pupilSizeL_filter.values]
        pupilSizeR_filterList = [i[0] for i in pupilSizeR_filter.values]
        
        #print('filter', len(pupilSizeL_filterList), len(pupilSizeR_filterList))
        RLCorrelation = np.corrcoef(pupilSizeL_filterList, pupilSizeR_filterList)
        #print(RLCorrelation)
        
        pupilAvg = [((pupilSizeL_filterList[i] + pupilSizeR_filterList[i])/2) for i in range(0, min(len(pupilSizeL_filterList), len(pupilSizeR_filterList)))]
        
        # First find baseline pupil size, which is the time when looking at NextPhrase key
        Samples_ForBaseline = int((int(DwellTimes_ForBaseline[trialNr-1][:-2])*90)/1000)+int((TimeFixation*900)/1000) # Number of samples of looking at key depend on
        
        #print(DwellTimes_ForBaseline[trialNr-1])
        
        # dwell time
        pupilL_baseline = np.mean(pupilSizeL_filterList[0:Samples_ForBaseline])
        pupilR_baseline = np.mean(pupilSizeR_filterList[0:Samples_ForBaseline])
        
        pupilL_Relative = [pupil - pupilL_baseline for pupil in pupilSizeL_filterList]
        pupilR_Relative = [pupil - pupilR_baseline for pupil in pupilSizeR_filterList]
        
        pupilRelativeAvg = [((pupilL_Relative[i] + pupilR_Relative[i])/2) for i in range(0, min(len(pupilL_Relative), len(pupilR_Relative)))]
        
        
        pupilLog_filter_wTime_Tuple = list(zip(time_filter, pupilAvg))
        pupilAndTimeDf =  pd.DataFrame(pupilLog_filter_wTime_Tuple, columns=['timestamp','pupildata'])
        
        # compute IPA for the trial
        ipaVal, coeff_modmax, coeff_hard, coeff_D2, coeff_D1, coeff_A, timePeriodTrial = ipaFunc(pupilAndTimeDf)
        
        #print(trialNr+1, ipaVal, timeOfGaze_Trial[-1])
        
        ipaList.append(ipaVal)
        timeOfGaze_TrialList.append(timeOfGaze_Trial[-1])
        
        
        blinkDurationList.append(blinkDuration)
        blinkFrequencyList.append(blinkFrequency)
        
        
        if len(blinkDuration)>0:
            blinkDurationAverageList.append(np.mean(blinkDuration))
        else:
            blinkDurationAverageList.append(0)
         
        pupilMean.append(np.mean(pupilRelativeAvg))
        
    return ipaList, pupilMean, timeOfGaze_TrialList, blinkDurationList, blinkDurationAverageList, blinkFrequencyList, timeInS_List

In [107]:
blink_wChinRestFrequency = list()
blink_wChinRestDurationAverage = list()

time_wChinRestTrial = list()
blink_wChinRestDuration = list()
ipa_wChinRest = list()
pupilMean_wChinRest = list()

score_wChinRestLIX = list()
score_wChinRestComplexity = list()
score_wChinRestDifficulty = list()
score_wChinRestSumOfScores = list()
score_wChinRestAfinn = list()

blink_woChinRestFrequency = list()
blink_woChinRestDurationAverage = list()

time_woChinRestTrial = list()
blink_woChinRestDuration = list()
ipa_woChinRest = list()
pupilMean_woChinRest = list()

score_woChinRestLIX = list()
score_woChinRestComplexity = list()
score_woChinRestDifficulty = list()
score_woChinRestSumOfScores = list()
score_woChinRestAfinn = list()

subjName = r'C:\DTU\Data\201812_ExptToCheckMovementEffect\Data'
j = 0
flagFirstSubj = 0
pupilData = dict()
pupilData['RLCorrelation'] = []

# extract self-reported scores list and LIX score of given sentence
file_name = r'C:/DTU/Data/201812_ExptToCheckMovementEffect/Data/Scores.xlsx'


#testNr = 'Test_wChinRest'
for root, dirs, subfolder in os.walk(subjName):
    scoreDifficult = list()
    scoreMedium = list()
    scoreEasy = list()
    # Semantic modeling score from afinn
    afinnDifficult = list()
    afinnMedium = list()
    afinnEasy = list()

    if not dirs:
        
        if 'tb' in root or 'trial' in root:
            continue
            
        userKeys = None
        gazeLog = None
        keysSelected = None
        
       
        
        for file in subfolder:
            if fnmatch.fnmatch(file, 'user_looks*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            
            elif fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
            
            elif fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])

                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or keysSelected is None or gazeLog is None:
                continue
            else:
                
                a = re.compile('(?<=ExptToCheckMovementEffect\\\\Data\\\\)(.*)(?=\\\\2018-1)')
                subjName = a.findall(root)[0]
                print('subject', subjName)
                
                sheet_to_df_map = pd.read_excel(file_name, sheet_name=subjName[0:2])
                
                testNr_re = re.compile('(?<=\\\Test)(.*)')
                testNr = 'Test' + testNr_re.findall(subjName)[0]
                if 'rh\Test_wChinRest' in subjName or 'sa\Test_woChinRest' in subjName:
                    testNr = 'Test' + testNr_re.findall(subjName)[0][:-3]
                print(testNr)
                columnName1 = testNr + '_SumOfScores'
                columnName2 = testNr + '_LIX'
                columnName3 = testNr + '_Complexity'
                columnName4 = testNr + '_Difficulty'
                columnName5 = testNr + '_AfinnScore'
                
                
                # find _wChinRest of difficulty score
                scoresSumOfScores = sheet_to_df_map[columnName1]
                scoresSumOfScores = scoresSumOfScores[1:]

                # find _wChinRest of SumOfScores score
                scoresLIX = sheet_to_df_map[columnName2]
                scoresLIX = scoresLIX[1:]
                
                # find _wChinRest of SumOfScores score
                scoresComplexity = sheet_to_df_map[columnName3]
                scoresComplexity = scoresComplexity[1:]
                
                # find _wChinRest of SumOfScores score
                scoresDifficulty = sheet_to_df_map[columnName4]
                scoresDifficulty = scoresDifficulty[1:]
                
                # find affin score
                scoresAfinn = sheet_to_df_map[columnName5]
                scoresAfinn = scoresAfinn[1:]
                
                if subjName == 'sa\Test_woChinRest\p2':
                    userKeys = userKeys[:-1]
                    
                # fix userKeys due to comma related file changes
                userKeys_new = FixUserKeys(userKeys)
                
                # find dwell time of typing
                userKeys_wDwellTime = ComputeDwellTime(userKeys_new)
                
                # find start time of typing
                timeTyping = OptiKeyTypingTime(userKeys_new)
                
                # for some of the subjects, the data was not completely collected
                if subjName == 'sa\Test_woChinRest\p1' or subjName == 'rh\Test_wChinRest\p1':
                    del keysSelected[-1]
                
                # divide complete data into epochs of phrases
                timeStartEndMixed = FindTrialEndTimes(keysSelected, timeTyping)
                
                # create trial time epoch using the list of start/end times of trial and userKeys, to make sure that 
                # Sleep is completely there in every trial, to allow for baseline
                timeEpochTrial = CreateTimeEpochsOfTrials(timeStartEndMixed, userKeys_new)
                #print(timeEpochTrial)
                
                dwellTimes_ForBaseline = DwellTimeForBaseline(userKeys_wDwellTime)
                
                # find and plot pupil size for every trial
                ipaList, pupilMean, timeOfTrialList, blinkDuration, blinkDurationAverage, blinkFrequency, time_trialList = FindAndPlotPupilSizeForEpoch(gazeLog, timeEpochTrial, scoresSumOfScores, dwellTimes_ForBaseline)
                
                if 'sa\Test_woChinRest' in subjName or 'rh\Test_wChinRest' in subjName:
                    if 'p1' in root:
                        ipaList1 = ipaList
                        pupilMean1 = pupilMean
                        timeOfTrialList1 = timeOfTrialList
                        blinkDurationAverage1 = blinkDurationAverage
                        blinkFrequency1 = blinkFrequency
                        time1 = time_trialList
                        blinkDuration1 = blinkDuration
                        continue
                        
                        #print('1', ipaList1)
                    else:
                        if 'sa\Test_woChinRest' in subjName:
                            ipaList2 = ipaList[1:]
                            pupilMean2 = pupilMean[1:]
                            timeOfTrialList2 = timeOfTrialList[1:]
                            blinkDurationAverage2 = blinkDurationAverage[1:]
                            blinkFrequency2 = blinkFrequency[1:]
                            time2 = time_trialList[1:]
                            blinkDuration2 = blinkDuration[1:]
                        else:
                            ipaList2 = ipaList
                            pupilMean2 = pupilMean
                            timeOfTrialList2 = timeOfTrialList
                            blinkDurationAverage2 = blinkDurationAverage
                            blinkFrequency2 = blinkFrequency
                            time2 = time_trialList
                            blinkDuration2 = blinkDuration
                            
                    blinkFrequency = blinkFrequency1 + blinkFrequency2
                    blinkDurationAverage = blinkDurationAverage1 + blinkDurationAverage2
                    time_trialList = time1 + time2
                    blinkDuration = blinkDuration1 + blinkDuration2
                    ipaList = ipaList1 + ipaList2
                    timeOfTrialList = timeOfTrialList1 + timeOfTrialList2
                    pupilMean = pupilMean1 + pupilMean2
                    
                timeOfTrialListPlot = [i/1000 for i in timeOfTrialList]


                if 'Test_wChinRest' in subjName:
                    blink_wChinRestFrequency.append(blinkFrequency)
                    blink_wChinRestDurationAverage.append(blinkDurationAverage)
                    time_wChinRestTrial.append(time_trialList)
                    blink_wChinRestDuration.append(blinkDuration)
                    ipa_wChinRest.append(ipaList)
                    pupilMean_wChinRest.append(pupilMean)
                    
                    score_wChinRestLIX.append(scoresLIX)
                    score_wChinRestComplexity.append(scoresComplexity)
                    score_wChinRestDifficulty.append(scoresDifficulty)
                    score_wChinRestSumOfScores.append(scoresSumOfScores)
                    score_wChinRestAfinn.append(scoresAfinn)
                else:
                    blink_woChinRestFrequency.append(blinkFrequency)
                    blink_woChinRestDurationAverage.append(blinkDurationAverage)
                    time_woChinRestTrial.append(time_trialList)
                    blink_woChinRestDuration.append(blinkDuration)
                    ipa_woChinRest.append(ipaList)
                    pupilMean_woChinRest.append(pupilMean)
                    
                    score_woChinRestLIX.append(scoresLIX)
                    score_woChinRestComplexity.append(scoresComplexity)
                    score_woChinRestDifficulty.append(scoresDifficulty)
                    score_woChinRestSumOfScores.append(scoresSumOfScores)
                    score_woChinRestAfinn.append(scoresAfinn)
                

subject bh\Test_wChinRest
Test_wChinRest
trial number  11 with 4.596693 s will be skipped
subject bh\Test_woChinRest
Test_woChinRest
trial number  11 with 7.339166 s will be skipped
subject ph\Test_wChinRest
Test_wChinRest
trial number  11 with 18.00399 s will be skipped
subject ph\Test_woChinRest
Test_woChinRest
subject pt\Test_wChinRest
Test_wChinRest
subject pt\Test_woChinRest
Test_woChinRest
trial number  11 with 17.47867 s will be skipped
subject rh\Test_wChinRest\p1
Test_wChinRest
subject rh\Test_wChinRest\p2
Test_wChinRest
trial number  4 with 10.770031 s will be skipped
trial number  10 with 14.989218 s will be skipped
subject rh\Test_woChinRest
Test_woChinRest
trial number  11 with 10.476645 s will be skipped
subject sa\Test_wChinRest
Test_wChinRest
subject sa\Test_woChinRest\p1
Test_woChinRest
subject sa\Test_woChinRest\p2
Test_woChinRest
trial number  3 with 8.449478 s will be skipped
trial number  5 with 8.926912 s will be skipped


In [108]:
scoreComplexity_wChinRest_difficult = list()
scoreDifficulty_wChinRest_difficult = list()
scoreAfinn_wChinRest_difficult = list()
scoreLIX_wChinRest_difficult = list()
            
blinkFrequency_wChinRest_difficult = list()
blinkDurationAverage_wChinRest_difficult = list()
            
ipa_wChinRest_difficult = list()
pupilMean_wChinRest_difficult = list()
            
scoreComplexity_wChinRest_easy = list()
scoreDifficulty_wChinRest_easy = list()
scoreAfinn_wChinRest_easy = list()
scoreLIX_wChinRest_easy = list()
            
blinkFrequency_wChinRest_easy = list()
blinkDurationAverage_wChinRest_easy = list()
            
ipa_wChinRest_easy = list()
pupilMean_wChinRest_easy = list()


subjInd = -1
for subjScores in score_wChinRestLIX:
    subjInd = subjInd + 1
    
    scoreComplexity_subj_diffiult = list()
    scoreDifficulty_subj_difficult = list()
    scoreAfinn_subj_difficult = list()
    scoreLIX_subj_difficult = list()
    blinkFrequency_subj_difficult = list()
    blinkDurationAverage_subj_difficult = list()
    ipa_subj_difficult = list()
    pupilMean_subj_difficult = list()
    
    scoreComplexity_subj_easy = list()
    scoreDifficulty_subj_easy = list()
    scoreAfinn_subj_easy = list()
    scoreLIX_subj_easy = list()
    blinkFrequency_subj_easy = list()
    blinkDurationAverage_subj_easy = list()
    ipa_subj_easy = list()
    pupilMean_subj_easy = list()
    
    scoreInd = -1
    blinkDurationList = list()
    for score in subjScores:
        scoreInd = scoreInd + 1
        if score > 2:
            
            scoreComplexity_subj_diffiult.append(score_wChinRestComplexity[subjInd][scoreInd+1])
            scoreDifficulty_subj_difficult.append(score_wChinRestDifficulty[subjInd][scoreInd+1])
            scoreAfinn_subj_difficult.append(score_wChinRestAfinn[subjInd][scoreInd+1])
            scoreLIX_subj_difficult.append(score_wChinRestLIX[subjInd][scoreInd+1])
            
            blinkFrequency_subj_difficult.append(blink_wChinRestFrequency[subjInd][scoreInd])
            blinkDurationAverage_subj_difficult.append(blink_wChinRestDurationAverage[subjInd][scoreInd])
            
            ipa_subj_difficult.append(ipa_wChinRest[subjInd][scoreInd])
            pupilMean_subj_difficult.append(pupilMean_wChinRest[subjInd][scoreInd])
            
        elif score < 2:
            scoreComplexity_subj_easy.append(score_wChinRestComplexity[subjInd][scoreInd+1])
            scoreDifficulty_subj_easy.append(score_wChinRestDifficulty[subjInd][scoreInd+1])
            scoreAfinn_subj_easy.append(score_wChinRestAfinn[subjInd][scoreInd+1])
            scoreLIX_subj_easy.append(score_wChinRestLIX[subjInd][scoreInd+1])
            
            blinkFrequency_subj_easy.append(blink_wChinRestFrequency[subjInd][scoreInd])
            blinkDurationAverage_subj_easy.append(blink_wChinRestDurationAverage[subjInd][scoreInd])
            
            ipa_subj_easy.append(ipa_wChinRest[subjInd][scoreInd])
            pupilMean_subj_easy.append(pupilMean_wChinRest[subjInd][scoreInd])
        
        if scoreInd == len(subjScores)-1:
            scoreComplexity_wChinRest_difficult.append(np.mean(scoreComplexity_subj_diffiult))
            scoreDifficulty_wChinRest_difficult.append(np.mean(scoreDifficulty_subj_difficult))
            scoreAfinn_wChinRest_difficult.append(np.mean(scoreAfinn_subj_difficult))
            scoreLIX_wChinRest_difficult.append(np.mean(scoreLIX_subj_difficult))
            
            blinkFrequency_wChinRest_difficult.append(np.mean(blinkFrequency_subj_difficult))
            blinkDurationAverage_wChinRest_difficult.append(np.mean(blinkDurationAverage_subj_difficult))
            
            ipa_wChinRest_difficult.append(np.mean(ipa_subj_difficult))
            pupilMean_wChinRest_difficult.append(np.mean(pupilMean_subj_difficult))
            
            scoreComplexity_wChinRest_easy.append(np.mean(scoreComplexity_subj_easy))
            scoreDifficulty_wChinRest_easy.append(np.mean(scoreDifficulty_subj_easy))
            scoreAfinn_wChinRest_easy.append(np.mean(scoreAfinn_subj_easy))
            scoreLIX_wChinRest_easy.append(np.mean(scoreLIX_subj_easy))
            
            blinkFrequency_wChinRest_easy.append(np.mean(blinkFrequency_subj_easy))
            blinkDurationAverage_wChinRest_easy.append(np.mean(blinkDurationAverage_subj_easy))
            
            ipa_wChinRest_easy.append(np.mean(ipa_subj_easy))
            pupilMean_wChinRest_easy.append(np.mean(pupilMean_subj_easy))
            
            

In [109]:
print(stats.ttest_rel(scoreComplexity_wChinRest_difficult, scoreComplexity_wChinRest_easy))
print(stats.ttest_rel(scoreDifficulty_wChinRest_difficult, scoreDifficulty_wChinRest_easy))
print(stats.ttest_rel(scoreAfinn_wChinRest_difficult, scoreAfinn_wChinRest_easy))

print(stats.ttest_rel(blinkFrequency_wChinRest_difficult, blinkFrequency_wChinRest_easy))
print(stats.ttest_rel(blinkDurationAverage_wChinRest_difficult, blinkDurationAverage_wChinRest_easy))

print(stats.ttest_rel(ipa_wChinRest_difficult, ipa_wChinRest_easy))
print(stats.ttest_rel(pupilMean_wChinRest_difficult, pupilMean_wChinRest_easy))


Ttest_relResult(statistic=1.49071198499986, pvalue=0.21029538311834686)
Ttest_relResult(statistic=2.502022303445038, pvalue=0.06662242870964402)
Ttest_relResult(statistic=-0.30151134457776363, pvalue=0.7780495489000209)
Ttest_relResult(statistic=0.4423763690161807, pvalue=0.681084280142571)
Ttest_relResult(statistic=0.24252793658836114, pvalue=0.8202991413081696)
Ttest_relResult(statistic=-0.5457914377870493, pvalue=0.6142206418288512)
Ttest_relResult(statistic=1.5815734814207545, pvalue=0.18890684090763923)


In [110]:
scoreComplexity_woChinRest_difficult = list()
scoreDifficulty_woChinRest_difficult = list()
scoreAfinn_woChinRest_difficult = list()
scoreLIX_woChinRest_difficult = list()

blinkFrequency_woChinRest_difficult = list()
blinkDurationAverage_woChinRest_difficult = list()
            
ipa_woChinRest_difficult = list()
pupilMean_woChinRest_difficult = list()
            
scoreComplexity_woChinRest_easy = list()
scoreDifficulty_woChinRest_easy = list()
scoreAfinn_woChinRest_easy = list()
scoreLIX_woChinRest_easy = list()

blinkFrequency_woChinRest_easy = list()
blinkDurationAverage_woChinRest_easy = list()
            
ipa_woChinRest_easy = list()
pupilMean_woChinRest_easy = list()


subjInd = -1
for subjScores in score_woChinRestLIX:
    subjInd = subjInd + 1
    
    scoreComplexity_subj_diffiult = list()
    scoreDifficulty_subj_difficult = list()
    scoreAfinn_subj_difficult = list()
    scoreLIX_subj_difficult = list()
    blinkFrequency_subj_difficult = list()
    blinkDurationAverage_subj_difficult = list()
    ipa_subj_difficult = list()
    pupilMean_subj_difficult = list()
    
    scoreComplexity_subj_easy = list()
    scoreDifficulty_subj_easy = list()
    scoreAfinn_subj_easy = list()
    scoreLIX_subj_easy = list()
    blinkFrequency_subj_easy = list()
    blinkDurationAverage_subj_easy = list()
    ipa_subj_easy = list()
    pupilMean_subj_easy = list()
    
    scoreInd = -1
    blinkDurationList = list()
    for score in subjScores:
        scoreInd = scoreInd + 1
        if score > 2:
            
            scoreComplexity_subj_diffiult.append(score_woChinRestComplexity[subjInd][scoreInd+1])
            scoreDifficulty_subj_difficult.append(score_woChinRestDifficulty[subjInd][scoreInd+1])
            scoreAfinn_subj_difficult.append(score_woChinRestAfinn[subjInd][scoreInd+1])
            scoreLIX_subj_difficult.append(score_woChinRestLIX[subjInd][scoreInd+1])
            
            blinkFrequency_subj_difficult.append(blink_woChinRestFrequency[subjInd][scoreInd])
            blinkDurationAverage_subj_difficult.append(blink_woChinRestDurationAverage[subjInd][scoreInd])
            
            ipa_subj_difficult.append(ipa_woChinRest[subjInd][scoreInd])
            pupilMean_subj_difficult.append(pupilMean_woChinRest[subjInd][scoreInd])
            
        elif score < 2:
            scoreComplexity_subj_easy.append(score_woChinRestComplexity[subjInd][scoreInd+1])
            scoreDifficulty_subj_easy.append(score_woChinRestDifficulty[subjInd][scoreInd+1])
            scoreAfinn_subj_easy.append(score_woChinRestAfinn[subjInd][scoreInd+1])
            scoreLIX_subj_easy.append(score_woChinRestLIX[subjInd][scoreInd+1])
            
            blinkFrequency_subj_easy.append(blink_woChinRestFrequency[subjInd][scoreInd])
            blinkDurationAverage_subj_easy.append(blink_woChinRestDurationAverage[subjInd][scoreInd])
            
            ipa_subj_easy.append(ipa_woChinRest[subjInd][scoreInd])
            pupilMean_subj_easy.append(pupilMean_woChinRest[subjInd][scoreInd])
        
        if scoreInd == len(subjScores)-1:
            scoreComplexity_woChinRest_difficult.append(np.mean(scoreComplexity_subj_diffiult))
            scoreDifficulty_woChinRest_difficult.append(np.mean(scoreDifficulty_subj_difficult))
            scoreAfinn_woChinRest_difficult.append(np.mean(scoreAfinn_subj_difficult))
            scoreLIX_woChinRest_difficult.append(np.mean(scoreLIX_subj_difficult))
            
            blinkFrequency_woChinRest_difficult.append(np.mean(blinkFrequency_subj_difficult))
            blinkDurationAverage_woChinRest_difficult.append(np.mean(blinkDurationAverage_subj_difficult))
            
            ipa_woChinRest_difficult.append(np.mean(ipa_subj_difficult))
            pupilMean_woChinRest_difficult.append(np.mean(pupilMean_subj_difficult))
            
            scoreComplexity_woChinRest_easy.append(np.mean(scoreComplexity_subj_easy))
            scoreDifficulty_woChinRest_easy.append(np.mean(scoreDifficulty_subj_easy))
            scoreAfinn_woChinRest_easy.append(np.mean(scoreAfinn_subj_easy))
            scoreLIX_woChinRest_easy.append(np.mean(scoreLIX_subj_easy))
            
            blinkFrequency_woChinRest_easy.append(np.mean(blinkFrequency_subj_easy))
            blinkDurationAverage_woChinRest_easy.append(np.mean(blinkDurationAverage_subj_easy))
            
            ipa_woChinRest_easy.append(np.mean(ipa_subj_easy))
            pupilMean_woChinRest_easy.append(np.mean(pupilMean_subj_easy))
            
            

In [111]:
print(stats.ttest_rel(scoreComplexity_woChinRest_difficult, scoreComplexity_woChinRest_easy))
print(stats.ttest_rel(scoreDifficulty_woChinRest_difficult, scoreDifficulty_woChinRest_easy))
print(stats.ttest_rel(scoreAfinn_woChinRest_difficult, scoreAfinn_woChinRest_easy))

print(stats.ttest_rel(blinkFrequency_woChinRest_difficult, blinkFrequency_woChinRest_easy))
print(stats.ttest_rel(blinkDurationAverage_woChinRest_difficult, blinkDurationAverage_woChinRest_easy))

print(stats.ttest_rel(ipa_woChinRest_difficult, ipa_woChinRest_easy))
print(stats.ttest_rel(pupilMean_woChinRest_difficult, pupilMean_woChinRest_easy))


Ttest_relResult(statistic=10.06230589874906, pvalue=0.0005486474801839636)
Ttest_relResult(statistic=9.130087106158284, pvalue=0.0007985330691842276)
Ttest_relResult(statistic=2.2499999999999996, pvalue=0.08764517650339469)
Ttest_relResult(statistic=2.8772898324005354, pvalue=0.045136030336156444)
Ttest_relResult(statistic=1.0745675745448307, pvalue=0.3430680792342676)
Ttest_relResult(statistic=-3.313182009310074, pvalue=0.029566315355539524)
Ttest_relResult(statistic=2.1219812269714295, pvalue=0.10111623933265561)


In [112]:
scoreComplexity_difficult = scoreComplexity_woChinRest_difficult + scoreComplexity_wChinRest_difficult
scoreDifficulty_difficult = scoreDifficulty_woChinRest_difficult + scoreDifficulty_wChinRest_difficult
scoreAfinn_difficult = scoreAfinn_woChinRest_difficult + scoreAfinn_wChinRest_difficult
scoreLIX_difficult = scoreLIX_wChinRest_difficult + scoreLIX_woChinRest_difficult
blinkFrequency_difficult = blinkFrequency_woChinRest_difficult + blinkFrequency_wChinRest_difficult
blinkDurationAverage_difficult = blinkDurationAverage_woChinRest_difficult + blinkDurationAverage_wChinRest_difficult
ipa_difficult = ipa_woChinRest_difficult + ipa_wChinRest_difficult
pupilMean_difficult = pupilMean_woChinRest_difficult + pupilMean_wChinRest_difficult


In [113]:
scoreComplexity_easy = scoreComplexity_woChinRest_easy + scoreComplexity_wChinRest_easy
scoreDifficulty_easy = scoreDifficulty_woChinRest_easy + scoreDifficulty_wChinRest_easy
scoreAfinn_easy = scoreAfinn_woChinRest_easy + scoreAfinn_wChinRest_easy
scoreLIX_easy = scoreLIX_woChinRest_easy + scoreLIX_wChinRest_easy

blinkFrequency_easy = blinkFrequency_woChinRest_easy + blinkFrequency_wChinRest_easy
blinkDurationAverage_easy = blinkDurationAverage_woChinRest_easy + blinkDurationAverage_wChinRest_easy
ipa_easy = ipa_woChinRest_easy + ipa_wChinRest_easy
pupilMean_easy = pupilMean_woChinRest_easy + pupilMean_wChinRest_easy


In [114]:
print(stats.ttest_rel(scoreComplexity_difficult, scoreComplexity_easy))
print(stats.ttest_rel(scoreDifficulty_difficult, scoreDifficulty_easy))
print(stats.ttest_rel(scoreAfinn_difficult, scoreAfinn_easy))

print(stats.ttest_rel(blinkFrequency_difficult, blinkFrequency_easy))
print(stats.ttest_rel(blinkDurationAverage_difficult, blinkDurationAverage_easy))

print(stats.ttest_rel(ipa_difficult, ipa_easy))
print(stats.ttest_rel(pupilMean_difficult, pupilMean_easy))


Ttest_relResult(statistic=4.133991732024803, pvalue=0.0025443073595036274)
Ttest_relResult(statistic=5.676263889047659, pvalue=0.0003033015559492453)
Ttest_relResult(statistic=0.5518254055364693, pvalue=0.594502686775326)
Ttest_relResult(statistic=1.6833457357225243, pvalue=0.12659779923829523)
Ttest_relResult(statistic=1.0725247044177222, pvalue=0.3113994960516582)
Ttest_relResult(statistic=-2.0667311720247867, pvalue=0.06872696101507828)
Ttest_relResult(statistic=2.2519398408807096, pvalue=0.05084186464959069)


In [115]:
ipa = ipa_difficult + ipa_easy
pupilMean = pupilMean_difficult + pupilMean_easy

blinkFrequency = blinkFrequency_difficult + blinkFrequency_easy
blinkDurationAverage = blinkDurationAverage_difficult + blinkDurationAverage_easy

scoreComplexity = scoreComplexity_difficult + scoreComplexity_easy
scoreDifficulty = scoreDifficulty_difficult + scoreDifficulty_easy
scoreAfinn = scoreAfinn_difficult + scoreAfinn_easy

scoreLIX = scoreLIX_difficult + scoreLIX_easy

In [116]:
ipa_wChinRest = ipa_wChinRest_difficult + ipa_wChinRest_easy
pupilMean_wChinRest = pupilMean_wChinRest_difficult + pupilMean_wChinRest_easy

blinkFrequency_wChinRest = blinkFrequency_wChinRest_difficult + blinkFrequency_wChinRest_easy
blinkDurationAverage_wChinRest = blinkDurationAverage_wChinRest_difficult + blinkDurationAverage_wChinRest_easy

scoreComplexity_wChinRest = scoreComplexity_wChinRest_difficult + scoreComplexity_wChinRest_easy
scoreDifficulty_wChinRest = scoreDifficulty_wChinRest_difficult + scoreDifficulty_wChinRest_easy
scoreAfinn_wChinRest = scoreAfinn_wChinRest_difficult + scoreAfinn_wChinRest_easy

scoreLIX_wChinRest = scoreLIX_wChinRest_difficult + scoreLIX_wChinRest_easy

In [117]:
ipa_woChinRest = ipa_woChinRest_difficult + ipa_woChinRest_easy
pupilMean_woChinRest = pupilMean_woChinRest_difficult + pupilMean_woChinRest_easy

blinkFrequency_woChinRest = blinkFrequency_woChinRest_difficult + blinkFrequency_woChinRest_easy
blinkDurationAverage_woChinRest = blinkDurationAverage_woChinRest_difficult + blinkDurationAverage_woChinRest_easy

scoreComplexity_woChinRest = scoreComplexity_woChinRest_difficult + scoreComplexity_woChinRest_easy
scoreDifficulty_woChinRest = scoreDifficulty_woChinRest_difficult + scoreDifficulty_woChinRest_easy
scoreAfinn_woChinRest = scoreAfinn_woChinRest_difficult + scoreAfinn_woChinRest_easy

scoreLIX_woChinRest = scoreLIX_woChinRest_difficult + scoreLIX_woChinRest_easy

In [118]:
print(len(pupilMean), len(ipa), len(blinkFrequency), len(y), len(blinkDurationAverage))

20 20 20 10 20


In [132]:
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

y = [0 if score == 1 else 1 for score in scoreLIX]

X = pd.DataFrame(
    {#'blinkFrequency': blinkFrequency,
     #'blinkDuration': blinkDurationAverage,
     #'scoreComplexity': scoreComplexity,
     #'scoreDifficulty': scoreDifficulty,
     #'scoreAfin' : scoreAfinn,
     #'ipa' : ipa,
     'pupilMean' : pupilMean
        
     
    })


logit_model=sm.Logit(y,X)
result=logit_model.fit(method='bfgs')

print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.676699
         Iterations: 5
         Function evaluations: 8
         Gradient evaluations: 8
                        Results: Logit
Model:              Logit            Pseudo R-squared: 0.024  
Dependent Variable: y                AIC:              29.0680
Date:               2018-12-22 14:10 BIC:              30.0637
No. Observations:   20               Log-Likelihood:   -13.534
Df Model:           0                LL-Null:          -13.863
Df Residuals:       19               Scale:            1.0000 
Converged:          1.0000                                    
---------------------------------------------------------------
              Coef.   Std.Err.    z     P>|z|    [0.025  0.975]
---------------------------------------------------------------
pupilMean     1.8157    2.3189  0.7830  0.4336  -2.7292  6.3607



In [120]:
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

y = [0 if score == 1 else 1 for score in scoreLIX_wChinRest]

X = pd.DataFrame(
    {'blinkFrequency': blinkFrequency_wChinRest,
     'blinkDuration': blinkDurationAverage_wChinRest,
     'scoreComplexity': scoreComplexity_wChinRest,
     'scoreDifficulty': scoreDifficulty_wChinRest,
     'scoreAfin' : scoreAfinn_wChinRest,
     'ipa' : ipa_wChinRest,
     'pupilMean' : pupilMean_wChinRest
        
     
    })


logit_model=sm.Logit(y,X)
result=logit_model.fit()

print(result.summary2())

         Current function value: inf
         Iterations: 35


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1264: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


LinAlgError: Singular matrix

In [121]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=109) # 70% training and 30% test

#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [122]:
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

Precision: 1.0
Recall: 1.0


In [123]:
plt.scatter(X.pupilMean,y)
plt.show()

In [124]:
X.pupilMean

0   -0.269275
1    0.135525
2    0.050791
3    0.160896
4    0.248927
5   -0.604614
6    0.003908
7    0.044167
8    0.168720
9    0.212247
Name: pupilMean, dtype: float64